### Step 0: Install autogluon via pip and import necessary packages 

Make sure you select the GPU accelerator

In [1]:
%%capture
# !pip install autogluon.vision --pre
# !pip install mxnet-cu110

In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import logging
logging.basicConfig(level=logging.INFO)
# attach logging to console
logging.getLogger().addHandler(logging.StreamHandler())

# Enter the competition!
### Step 1: import autogluon

In [3]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/zv/7_nb_lnx5llfxwncr2x_6tvh0000gn/T/tmptmotmqyt
Created a temporary directory at /var/folders/zv/7_nb_lnx5llfxwncr2x_6tvh0000gn/T/tmptmotmqyt
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/zv/7_nb_lnx5llfxwncr2x_6tvh0000gn/T/tmptmotmqyt/_remote_module_non_scriptable.py
Writing /var/folders/zv/7_nb_lnx5llfxwncr2x_6tvh0000gn/T/tmptmotmqyt/_remote_module_non_scriptable.py


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /Users/frank/anaconda3/envs/d2l-zh/lib/python3.9/site-packages/aiohttp/client_reqrep.py:70 in    │
│ <module>                                                                                         │
│                                                                                                  │
│     67 │   SSLContext = object  # type: ignore[misc,assignment]                                  │
│     68                                                                                           │
│     69 try:                                                                                      │
│ ❱   70 │   import cchardet as chardet                                                            │
│     71 except ImportError:  # pragma: no cover                                                   │
│     72 │   import charset_normalizer as chardet  # type: ignore[no-redef]                        │
│     73                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'cchardet'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 import autogluon.core as ag                                                                  │
│ ❱ 2 from autogluon.vision import ImagePredictor                                                  │
│   3                                                                                              │
│                                                                                                  │
│ /Users/frank/anaconda3/envs/d2l-zh/lib/python3.9/site-packages/autogluon/vision/__init__.py:1 in │
│ <module>                                                                                         │
│                                                                                                  │
│ ❱ 1 from .predictor import ImagePredictor                                                        │
│   2 from .detector import ObjectDetector                                                         │
│   3                                                                                              │
│   4 ImageDataset = ImagePredictor.Dataset                                                        │
│                                                                                                  │
│ /Users/frank/anaconda3/envs/d2l-zh/lib/python3.9/site-packages/autogluon/vision/predictor/__init │
│ __.py:1 in <module>                                                                              │
│                                                                                                  │
│ ❱ 1 from .predictor import *                                                                     │
│   2                                                                                              │
│                                                                                                  │
│ /Users/frank/anaconda3/envs/d2l-zh/lib/python3.9/site-packages/autogluon/vision/predictor/predic │
│ tor.py:22 in <module>                                                                            │
│                                                                                                  │
│    19 from autogluon.core.data.label_cleaner import LabelCleaner                                 │
│    20 from autogluon.core.utils.try_import import try_import_d8                                  │
│    21 from autogluon.core.utils.utils impo

### Step 2: Apply customization to competition data

In [ ]:
def fix_root(df, root='/kaggle/input/classify-leaves'):
    df['image'] = df['image'].apply(lambda x: os.path.join(root, x))
    return df

# load train data
train_data = fix_root(pd.read_csv('/kaggle/input/classify-leaves/train.csv'))

In [ ]:
predictor = ImagePredictor(verbosity=1).fit(train_data, time_limit=3600*3, presets='medium_quality_faster_train', hyperparameters={'epochs': 30, 'model': 'resnest50', 'lr': ag.Categorical(0.1, 0.15)}) 
print(predictor.fit_summary())

### Step 3: Test

In [ ]:
test = pd.read_csv('/kaggle/input/classify-leaves/test.csv')
test

In [ ]:
pred_test = predictor.predict(fix_root(test.copy()))
# save prediction with image id
y = pd.concat([test, pred_test.to_frame(name='label')], axis=1)
y.to_csv('submission_test.csv', index=False)
y

### Last step: save model and submission file

In [ ]:
predictor.save('predictor.ag')